In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 58.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 120.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.1 MB/s eta 0:00:00


In [ ]:
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.4/492.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.7 MB/s eta 0:00:00


In [ ]:
import numpy as np
import torch
import cv2
import torch.nn as nn
from transformers import ViTModel,ViTConfig,ViTFeatureExtractor
from torchvision import transforms
from torch.optim import Adam
from torch.utils.data import DataLoader, SubsetRandomSampler,RandomSampler
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
checkpoint='google/vit-base-patch16-224-in21k'

In [96]:
seed=42
G=torch.Generator()
G.manual_seed(seed)

In [ ]:
from datasets import load_dataset

dataset=load_dataset('Matthijs/snacks')
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 4838
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 952
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 955
    })
})


In [154]:
#Now writing a dataset for this dataset
import torch.nn as nn

class Snacks_Dataset(torch.utils.data.Dataset):
  def __init__(self,dataset,transform1=None,transform2=None):
    self.dataset=dataset
    self.transform1=transform1
    self.transform2=transform2
  def __len__(self):
    return len(self.dataset)
  def __getitem__(self,idx):
    image=self.dataset[idx]['image']
    labels=self.dataset[idx]['label']

    if self.transform1!=None:
      image1=self.transform1(image)
    if self.transform2!=None:
      image2=self.transform2(image)

    return image1,image2,labels

In [155]:
from torchvision import transforms

transform=transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Resize((224,224),antialias=True),
        transforms.Normalize(mean=[0.5,0.5,0.5],
                             std=[0.5,0.5,0.5]
                             )
    ]
)

In [156]:
from torchvision import transforms

transform1=transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Resize((299,299),antialias=True),
        transforms.Normalize(mean=[0.5,0.5,0.5],
                             std=[0.5,0.5,0.5]
                             )
    ]
)

In [157]:
trainset=Snacks_Dataset(dataset['train'],transform,transform1)
validset=Snacks_Dataset(dataset['validation'],transform,transform1)
testset=Snacks_Dataset(dataset['test'],transform,transform1)

In [158]:
print(f"Length of the Training Set {len(trainset)}")
print(f"Length of the Validation Set {len(validset)}")
print(f"Length of the Test Set {len(testset)}")

Length of the Training Set 4838
Length of the Validation Set 955
Length of the Test Set 952


In [159]:
print(trainset[0][2])

0


In [192]:
trainloader=torch.utils.data.DataLoader(trainset,batch_size=16,shuffle=True)
validloader=torch.utils.data.DataLoader(validset,batch_size=16,shuffle=True)

In [161]:
for data in trainloader:
  image,image1,label=data
  break

print(f"One batch image size: {image.shape} ")
print(f"one another image size:{image1.shape}")
print(f"One batch label size: {label.shape}")

One batch image size: torch.Size([16, 3, 224, 224]) 
one another image size:torch.Size([16, 3, 299, 299])
One batch label size: torch.Size([16])


In [162]:
class SnackVit(nn.Module):
  def __init__(self,config=ViTConfig(),num_labels=20,checkpoint=checkpoint):
    super(SnackVit,self).__init__()
    self.vit=ViTModel.from_pretrained(checkpoint,add_pooling_layer=False)

    self.classifier=nn.Linear(config.hidden_size,num_labels)

  def forward(self,image,check=False):
    x=self.vit(image)['last_hidden_state']
    if check==True:
      print('shape is: ',x[:,0,:].shape)
      return x[:,0,:]
    out=self.classifier(x[:,0,:])

    return out

# VIT Model steps

In [163]:
model=SnackVit()
model.to('cuda')

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTModel: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing ViTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


SnackVit(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=768, out_features=

In [207]:
def train_function(model,trainloader,optimizer,model_name=None):
  train_lossv=0.0
  train_accv=0.0
  train_lossc=0.0
  train_accc=0.0
  model[0].train()
  model[1].train()
  for data in tqdm(trainloader):
     vimages,cimages,keys=data
     vimages=vimages.to('cuda')
     cimages=cimages.to('cuda')
     keys=keys.to('cuda')
     logitsv=model[0](vimages,False)
     logitsc=model[1](cimages,False)
     lossv=nn.CrossEntropyLoss()(logitsv,keys)
     lossc=nn.CrossEntropyLoss()(logitsc,keys)
     train_accv+=(logitsv.argmax(dim=1)==keys).sum().item()
     train_accc+=(logitsc.argmax(dim=1)==keys).sum().item()
     train_lossv+=lossv.item()
     train_lossc+=lossc.item()
     optimizer[0].zero_grad()
     lossv.backward()
     optimizer[0].step()
     optimizer[1].zero_grad()
     lossc.backward()
     optimizer[1].step()



  return train_lossv/len(trainloader.dataset),train_accv/len(trainloader.dataset),train_lossc/len(trainloader.dataset),train_accc/len(trainloader.dataset)

In [208]:
def valid_function(model,validloader,model_name=None):
  valid_lossv=0.0
  valid_accv=0.0
  valid_lossc=0.0
  valid_accc=0.0
  model[0].eval()
  model[1].eval()
  with torch.no_grad():
    for data in tqdm(validloader):
      vimages,cimages,keys=data
      vimages=vimages.to('cuda')
      cimages=cimages.to('cuda')
      keys=keys.to('cuda')

      logitsv=model[0](vimages,False)
      logitsc=model[1](cimages,False)

      lossv=nn.CrossEntropyLoss()(logitsv,keys)
      lossc=nn.CrossEntropyLoss()(logitsc,keys)
      valid_accv+=(logitsv.argmax(dim=1)== keys).sum().item()
      valid_accc+=(logitsc.argmax(dim=1)==keys).sum().item()
      valid_lossv+=lossv.item()
      valid_lossc=lossc.item()

    return valid_lossv/len(validloader.dataset),valid_accv/len(validloader.dataset),valid_lossc/len(validloader.dataset),valid_accc/len(validloader.dataset)

In [209]:
from torchvision import models

In [210]:
class CNNmodel(nn.Module):
  def __init__(self):
    super(CNNmodel,self).__init__()
    self.backbone=models.inception_v3(pretrained=True)
    self.backbone.aux_logits=False
    self.classifier=nn.Sequential(nn.Linear(self.backbone.fc.out_features,20))
  def forward(self,image,check=False):
    x=self.backbone(image)
    out=self.classifier(x)
    if check==True:
      return x
    return out

In [211]:
cnn=CNNmodel()
cnn.to('cuda');

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


# Training Section

In [212]:
epochs=5
optimizer=torch.optim.Adam(model.parameters(),lr=0.0001)
optimizer1=torch.optim.Adam(cnn.parameters(),lr=0.0001)
best_valid_accv=-99
best_valid_accc=-99
for i in range(epochs):
  train_lossv,train_accv,train_lossc,train_accc=train_function([model,cnn],trainloader,optimizer=[optimizer,optimizer1],model_name='vit')
  valid_lossv,valid_accv,valid_lossc,valid_accc=valid_function([model,cnn],validloader,model_name='vit')
  if valid_accv>best_valid_accv:
    torch.save(model.state_dict(),"Snack Model.pt")
    best_valid_accv=valid_accv
  if valid_accc>best_valid_accc:
    torch.save(model.state_dict(),"Snack cnn .pt")
    best_valid_accc=valid_accc
    print("Model Saved")
  print(f'Epoch{i+1}/{epochs} Transformer Training Accuracy {train_accv} Validation Acc: {valid_accv}')
  print(f'Epoch{i+1}/{epochs} CNN Testing Accuracy {valid_accv} Validation Acc: {valid_accc}')

100%|██████████| 60/60 [00:20<00:00,  2.97it/s]


Model Saved
Epoch1/5 Transformer Training Accuracy 0.9892517569243489 Validation Acc: 0.8952879581151832
Epoch1/5 CNN Testing Accuracy 0.8952879581151832 Validation Acc: 0.8198952879581152


100%|██████████| 60/60 [00:20<00:00,  2.90it/s]


Model Saved
Epoch2/5 Transformer Training Accuracy 0.9865646961554362 Validation Acc: 0.875392670157068
Epoch2/5 CNN Testing Accuracy 0.875392670157068 Validation Acc: 0.8439790575916231


100%|██████████| 60/60 [00:23<00:00,  2.50it/s]


Epoch3/5 Transformer Training Accuracy 0.9873914840843324 Validation Acc: 0.8691099476439791
Epoch3/5 CNN Testing Accuracy 0.8691099476439791 Validation Acc: 0.818848167539267


100%|██████████| 60/60 [00:22<00:00,  2.68it/s]


Epoch4/5 Transformer Training Accuracy 0.9863579991732121 Validation Acc: 0.8554973821989529
Epoch4/5 CNN Testing Accuracy 0.8554973821989529 Validation Acc: 0.8293193717277487


100%|██████████| 60/60 [00:20<00:00,  2.95it/s]

Epoch5/5 Transformer Training Accuracy 0.9869780901198842 Validation Acc: 0.8471204188481676
Epoch5/5 CNN Testing Accuracy 0.8471204188481676 Validation Acc: 0.8293193717277487


In [213]:
x=torch.randn(1,3,224,224).to(
    'cuda'
)
output=model(x,True)
print(output.shape)

shape is:  torch.Size([1, 768])
torch.Size([1, 768])


In [216]:
from itertools import chain
def training_feature_extractor(model,trainloader,send_label=False,model_name=None):
  x_train=[]
  y_train=[]

  for vimage,cimage,label in tqdm(trainloader):
      vimage=vimage.to('cuda')
      cimage=cimage.to('cuda')
      label=label.to('cuda')
      output1=model[0](vimage,True)
      output1=output1.detach().cpu().numpy()
      output2=model[1](cimage,True)
      output2=output2.detach().cpu().numpy()
      final_out=np.concatenate((output1,output2),axis=1)
      x_train.append(final_out)
      label=label.cpu().detach().numpy()
      label=label.astype('int')
      y_train.append(label)
      torch.cuda.empty_cache()

  x_train_features=list(chain.from_iterable(x_train))
  y_train=list(chain.from_iterable(y_train))
  print("Total Training elements: ",len(x_train_features))
  print("Total Training Labels: ",len(y_train))
  x_train_features = np.array(x_train_features)
  y_train=np.array(y_train)
  print("Shape of Training Labels: ",y_train.shape)
  if send_label==False:
   return x_train_features

  return x_train_features,y_train

In [217]:
def test_loader_feature_extraction(model,testloader=validloader,send_label=False,model_name=None):
  x_test=[]
  y_test=[]
  features={}
  for vimage,cimage,label in tqdm(testloader):
      vimage=vimage.to('cuda')
      cimage=cimage.to('cuda')
      label=label.to('cuda')
      output=model[0](vimage,True)

      output=output.detach().cpu().numpy()
      output1=model[1](cimage,True)
      output1=output1.detach().cpu().numpy()
      final_out=np.concatenate((output,output1),axis=1)
      x_test.append(final_out)
      label=label.cpu().detach().numpy()
      label=label.astype('int')
      y_test.append(label)
      torch.cuda.empty_cache()
  x_val_features=list(chain.from_iterable(x_test))
  y_val=list(chain.from_iterable(y_test))
  print("Total Training elements: ",len(x_val_features))
  print("Total Training Labels: ",len(y_val))
  x_val_features = np.array(x_val_features)
  print("Shape of Testing features: ",x_val_features.shape)
  y_val=np.array(y_val)
  print("Shape of Training Labels: ",y_val.shape)
  if send_label==False:
    return x_val_features
  return x_val_features,y_val

In [219]:
train_feature,train_labels=training_feature_extractor([model,cnn],trainloader,send_label=True,model_name='vit')

  0%|          | 0/303 [00:00<?, ?it/s]

shape is:  torch.Size([16, 768])


  0%|          | 1/303 [00:00<02:18,  2.18it/s]

shape is:  torch.Size([16, 768])


  1%|          | 2/303 [00:00<01:58,  2.53it/s]

...

shape is:  torch.Size([16, 768])


 99%|█████████▉| 301/303 [01:54<00:00,  2.82it/s]

shape is:  torch.Size([16, 768])


100%|██████████| 303/303 [01:55<00:00,  2.63it/s]

shape is:  torch.Size([6, 768])
Total Training elements:  4838
Total Training Labels:  4838
Shape of Training Labels:  (4838,)


In [220]:
test_feature,test_labels=test_loader_feature_extraction([model,cnn],validloader,send_label=True,model_name='vit')

  0%|          | 0/60 [00:00<?, ?it/s]

shape is:  torch.Size([16, 768])


  2%|▏         | 1/60 [00:00<00:26,  2.19it/s]

shape is:  torch.Size([16, 768])


  3%|▎         | 2/60 [00:00<00:24,  2.35it/s]

...

shape is:  torch.Size([16, 768])


 97%|█████████▋| 58/60 [00:21<00:00,  2.82it/s]

shape is:  torch.Size([16, 768])


100%|██████████| 60/60 [00:21<00:00,  2.73it/s]

shape is:  torch.Size([11, 768])
Total Training elements:  955
Total Training Labels:  955
Shape of Testing features:  (955, 1768)
Shape of Training Labels:  (955,)


# CNN Part

# CNN sec

In [221]:
from sklearn.preprocessing import StandardScaler

sc=StandardScaler()
X_train=sc.fit_transform(train_feature)
X_val=sc.transform(test_feature)

print("After scaling train: ", X_train.shape)
print("After scaling test size: ", X_val.shape)

After scaling train:  (4838, 1768)
After scaling test size:  (955, 1768)


In [224]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
mlp_classifier=MLPClassifier(hidden_layer_sizes=(3000,),learning_rate_init=0.0001,random_state=42,max_iter=2000000)

mlp_classifier.fit(X_train,train_labels)

predictions=mlp_classifier.predict(X_val)
val_acc=accuracy_score(test_labels,predictions)
print(val_acc)


0.8994764397905759


# MIG Feature Extraction

In [225]:
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectKBest

In [230]:
select=SelectKBest(mutual_info_classif,k=500)
X_train_mg=select.fit_transform(X_train,train_labels)
X_test_mg=select.transform(X_val)

In [232]:
mlp_classifier=MLPClassifier(hidden_layer_sizes=(100,),learning_rate_init=0.0001,random_state=42,max_iter=2000000)

mlp_classifier.fit(X_train_mg,train_labels)

predictions=mlp_classifier.predict(X_test_mg)
val_acc=accuracy_score(test_labels,predictions)
print(val_acc)

0.8827225130890053
